# Install Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/model2-007/architecture/final_preprocess.py /content

In [3]:
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/model2-007/architecture/final_utilities.py /content

In [4]:
%%capture
!pip install monai
!pip install dicom2nifti

In [32]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss

import torch
from final_utilities import train

import numpy as np
from skimage import exposure
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Resized,
    ToTensord,
)
from monai.data import CacheDataset, Dataset, DataLoader
from glob import glob
import os
from monai.utils import set_determinism

# Preprocessing Dataset

In [33]:
def preprocess_image(image, a_min=-200, a_max=200):
    adjusted_image = (image - 60 + 200) / 400
    normalized_image = (adjusted_image - np.min(adjusted_image)) / (np.max(adjusted_image) - np.min(adjusted_image))
    # convert the PyTorch tensor to a NumPy array
    normalized_image_np = normalized_image.numpy()
    enhanced_image = exposure.equalize_adapthist(normalized_image_np)
    return enhanced_image

def prepare(in_dir, pixdim=(1.5, 1.5, 1.0), a_min=-200, a_max=200, spatial_size=[128, 128, 64], cache=False):
    set_determinism(seed=0)

    def combined_transforms(data):

        enhanced_vol = preprocess_image(data["vol"], a_min=a_min, a_max=a_max)

        return {"vol": enhanced_vol, "seg": data["seg"]}

    path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))

    path_test_volumes = sorted(glob(os.path.join(in_dir, "TestVolumes", "*.nii.gz")))
    path_test_segmentation = sorted(glob(os.path.join(in_dir, "TestSegmentation", "*.nii.gz")))

    train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
    test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]

    combined_train_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),
            # Applying the combined preprocessing function here
            combined_transforms,
            ToTensord(keys=["vol", "seg"]),
        ]
    )

    combined_test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),
            # Applying the combined preprocessing function here
            combined_transforms,
            ToTensord(keys=["vol", "seg"]),
        ]
    )

    if cache:
        train_ds = CacheDataset(data=train_files, transform=combined_train_transforms, cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = CacheDataset(data=test_files, transform=combined_test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader

    else:
        train_ds = Dataset(data=train_files, transform=combined_train_transforms)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = Dataset(data=test_files, transform=combined_test_transforms)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader


In [34]:
data_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/model2-007/dataset-007/Data_Train_Test/'

## Checking for CUDA

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. You can use the GPU.")
else:
    print("CUDA is not available. You can only use the CPU.")

CUDA is available. You can use the GPU.


## Preprocess dataset through prepare function

In [10]:
data_in = prepare(data_dir, cache=True)

Loading dataset: 100%|██████████| 56/56 [01:54<00:00,  2.04s/it]


# Model Library

## 1. Classical Unet

In [ ]:
device = torch.device("cuda:0")

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
    dropout = 0.2
).to(device)

## 2. Unet with Batch and ReLU, Residual Units (Need to test)

In [ ]:
from monai.networks.layers.factories import Norm, Act

device = torch.device("cuda:0")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
    act=Act.RELU,
    dropout = 0.2
).to(device)

In [ ]:
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5, amsgrad=True)

## 3. Residual Unet

In [ ]:
from monai.networks.nets import DynUNet

device = torch.device("cuda:0")
model = DynUNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    kernel_size=(3, 3, 3, 3),
    strides=(1, 2, 2, 2),
    upsample_kernel_size=(2, 2, 2),
    res_block=True,
    dropout = 0.2
).to(device)

## 4. Residual Unet, Group Normalization - v5

In [25]:
model_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/results/model-007/v5'

In [28]:
from monai.networks.layers.factories import Norm, Act

device = torch.device("cuda:0")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=("group", {"num_groups": 2}),
    act=Act.RELU,
    dropout = 0.2
).to(device)

In [30]:
loss_function = DiceLoss(to_onehot_y=True, softmax=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5, amsgrad=True)

In [ ]:
if __name__ == '__main__':
    train(model, data_in, loss_function, optimizer, 150, model_dir)

## 5. AHNet

In [ ]:
model_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/results/model-007/v6'

In [39]:
from monai.networks.nets import AHNet

device = torch.device("cuda:0")
model = AHNet(
    layers= (3,4,6,3),
    spatial_dims= 3,
    in_channels= 1,
    out_channels= 2,
    psp_block_num=4,
    upsample_mode='transpose',
    pretrained= True
).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 303MB/s]


In [41]:
from torch import nn

loss_function = DiceLoss(to_onehot_y=True, softmax=True).to(device)
# loss_function = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, amsgrad=True)

In [ ]:
if __name__ == '__main__':
    train(model, data_in, loss_function, optimizer, 150, model_dir)

## Check the Size of Data (Add to main)

In [ ]:
# assuming prepare function has been imported from another file
train_loader, test_loader = prepare(data_dir)

for batch_data in train_loader:
  volume = batch_data["vol"]
  label = batch_data["seg"]

  # print size of volume and label tensors
  print(f"Volume size: {volume.size()}")
  print(f"Label size: {label.size()}")

  # only process one batch
  break